# 026 change in resolution for concatenation
* uses taxonomic information from GTDB (see 005.generate_gtdb_sheet.ipynb)


In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from Bio import Seq, SeqIO, Align, AlignIO, Phylo, Alphabet, pairwise2 
from Bio.SeqRecord import SeqRecord
from Bio.Align import AlignInfo, Applications
from Bio.Phylo import draw, TreeConstruction # TreeConstruction.DistanceTreeConstructor 
# https://bioinformatics.stackexchange.com/questions/4337/ \
#biopython-phylogenetic-tree-replace-branch-tip-labels-by-sequence-logos

import numpy as np
import seaborn as sns
from sklearn import manifold, metrics, cluster, neighbors, decomposition, preprocessing
import skbio, parasail, dendropy, pandas
import sys, gzip, re, glob, pickle, collections, subprocess, os, errno, random, itertools

def print_redblack(textr, textbb="", textbl=""):
    print ('\x1b[0;1;31;1m'+ str(textr) + '\x1b[0;1;30;1m'+ str(textbb) + '\x1b[0;0;30;0m'+ str(textbl) + '\x1b[0m')

## Some global variables
- where to save output, list of _arbitrarily_chosen genera (as well as list with all genera if you want to change it), RNA gene names

In [2]:
# this is the set of possibilities (that we previously downloaded from refseq) 
all_genera = ["Campylobacter","Enterococcus","Klebsiella","Listeria","Neisseria","Staphylococcus",
                 "Vibrio","Clostridium","Escherichia","Helicobacter","Leptospira","Mycobacterium",
                 "Pseudomonas","Salmonella","Streptococcus"] ## we don't use this variable in practice
# and these were _arbitrarily_ chosen, based on representativity (i.e. we excluded smaller sets like Lepto, Helico, 
# Clostridium, Vibrio). But you are free to try different combinations 
#chosen_genera = ["Enterococcus","Escherichia", "Pseudomonas","Salmonella","Staphylococcus","Streptococcus"]

## DEBUG

chosen_genera = ["Escherichia", "Pseudomonas","Staphylococcus"]

outdir = "./026_results/"
rrna_types = ["16S", "23S", "5S"];
hyper_types = ["16Sv1v2", "16Sv3v4"];
iupac_dna = {''.join(sorted(v)):k for k,v in Seq.IUPAC.IUPACData.ambiguous_dna_values.items()}

In [3]:
def species_list_from_binomial (binomial_str):
    return binomial_str.split()[:2]  ## two first names, since might have strain, serovar
    
def species_name_from_binomial (binomial_str, accession_str=None):
    binstr = ".".join(binomial_str.split()[:2]) ## join with \. since \_ is already used bt GTDB
    if (accession_str):
        accession_str = str(re.search('GCF_(\d+)\.', accession_str).group(1)) # gets number inside GCF_ddd.1
        binstr += "." + accession_str
    return binstr

def subsample_accept_genus (sp_name, list_of_genera = chosen_genera):
    for x in list_of_genera:
        if x in sp_name:
            return True
    return False

def subsample_accept_counter (sp_name, collection_counter):
    if sp_name not in collection_counter:
        return False
    x = collection_counter[sp_name]
    if x < 6:
        return False
    if x > 40:
        return np.random.random() < 40/x
    return True

## Read table with files and taxonomic classification
- This table was generated in R with notebook [005.generate_gtdb_sheet.ipynb](./005.generate_gtdb_sheet.ipynb) and contains taxonomies (SILVA, NCBI, and GTDB) for all valid genome files that we downloaded. (By 'valid' we mean those passing the quality checks of GTDB.)
- In cell below, we add first column as file name, so e.g. `gtdb_taxonomy` is `table[2]`

In [4]:
gbff_dir = "/media/deolivl/QIB_deolivl/bigdata/"   ## directory (or directories, in our case) with refseq genomes
#gbff_dir = "./bigdata/"

file_lines = [line.strip() for line in open("./bigdata/gtdb_list.csv", 'r')]
print_redblack("header: <file>  ", file_lines[0].split(',')[1:]) ## first column is data.frame index from R

table_filenames_species = [line.split(',')[1:] for line in file_lines[1:]] ## skip first line, with headers
print ("first element of list: ", table_filenames_species[0], "\n")

fnames = glob.glob(gbff_dir + "*/GCF_*.gbff.gz")
idx=[i for j in table_filenames_species for i,fname in enumerate(fnames) if j[0] in fname] ## j[1] has accession 
table_filenames_species = [[fnames[idx[i]]] + j for i,j in enumerate(table_filenames_species)]
print ("first element, with file location: ", table_filenames_species[0])

header: <file>  ['accession', 'gtdb_taxonomy', 'lsu_silva_23s_taxonomy', 'ncbi_organism_name', 'ncbi_taxonomy', 'ssu_silva_taxonomy']
first element of list:  ['GCF_000465235.1', 'Campylobacter_D coli', 'Campylobacter coli CVM N29710', 'Campylobacter coli CVM N29710', 'Campylobacter coli', 'Campylobacter jejuni 30318'] 

first element, with file location:  ['/media/deolivl/QIB_deolivl/bigdata/Campylobacter/GCF_000465235.1_ASM46523v1_genomic.gbff.gz', 'GCF_000465235.1', 'Campylobacter_D coli', 'Campylobacter coli CVM N29710', 'Campylobacter coli CVM N29710', 'Campylobacter coli', 'Campylobacter jejuni 30318']


## randomly choose samples from database
- subsample based on species representativity, i.e. species w/ more than 40 are downsampled (resulting with at most ~40 samples for each species)
- also limit to a few genera (_arbitrarily_ selected)
- result is stored in a table that contains both the file path, and several taxonomic inferences (from GTDB spreadsheet)

'species' follows the GTDB classification, BTW. 

In [5]:
print_redblack ("Full set of genera in GTDB:", "", set([x[2].split()[0] for x  in table_filenames_species]))
tbl_fl_sp = list(table_filenames_species)
# remove all samples that don't belong to set of _arbitrarily_ chosen genera
tbl_fl_sp = [x  for x in tbl_fl_sp if subsample_accept_genus (x[2])] 
species_counter = collections.Counter([x[2] for x in tbl_fl_sp if "sp." not in x[2]]) ## remove generic "sp."
print_redblack ("\nmost common species in selected genera: ", "", species_counter.most_common(10))

tbl_fl_sp = [x  for x in table_filenames_species if subsample_accept_counter (x[2], species_counter)]

tmp_counter = collections.Counter([x[2] for x in tbl_fl_sp])
print_redblack("\nFinal set with " + str(len(tbl_fl_sp)) + " samples ", "(" + str(len(tmp_counter))+ " species) ",tmp_counter)
random.shuffle(tbl_fl_sp)

Full set of genera in GTDB:{'Helicobacter_H', 'Clostridium', 'Leptospira', 'Neisseria', 'Pseudomonas_A', 'Clostridium_AM', 'Helicobacter_C', 'Neisseria_B', 'Helicobacter_E', 'Vibrio', 'Streptococcus', 'Clostridium_B', 'Clostridium_I', 'Clostridium_S', 'Enterococcus', 'Clostridium_H', 'Klebsiella', 'Ruminiclostridium_A', 'Clostridium_K', 'Mycobacterium', 'Mycolicibacterium', 'Clostridium_P', 'Helicobacter_D', 'Pseudomonas', 'Pseudomonas_E', 'Clostridium_AN', 'Klebsiella_B', 'Staphylococcus', 'Staphylococcus_A', 'Enterococcus_B', 'Enterococcus_E', 'Helicobacter_A', 'Campylobacter', 'Campylobacter_A', 'Cedecea', 'Helicobacter_G', 'UBA11063', 'Klebsiella_A', 'Leptospira_A', 'Clostridium_AD', 'Pseudomonas_D', 'Campylobacter_B', 'Listeria', 'Pseudomonas_B', 'Clostridium_F', 'Bergeriella', 'Clostridium_G', 'Escherichia', 'Clostridium_W', 'Pseudomonas_F', 'Helicobacter_F', 'Helicobacter', 'Neisseria_D', 'Salmonella', 'Clostridium_J', 'Eubacterium_I', 'Enterococcus_D', 'Mycobacteroides', 'Campy

## Auxiliary functions 
- alignment wrappers, choice of RNA sequence (when there are several copies)
- how to extract hypervariable regions from 16S: we use Smith-Waterman to find established primers, for v1v2 and v3v4:
>We used the primer pair 27 F/338 R (27 F: 5′-AGAGTTTGATCCTGGCTCAG-3′/ 338 R: 5′-TGCTGCCTCCCGTAGGAGT-3′) to amplify the >V1/V2 region, and the primer pair V3F/V4R (V3F: 5′-CCTACGGGAGGCAGCAG-3′/ V4R: 5′-GGACTACHVGGGTWTCTAAT-3′) for the V3/V4 >hypervariable region from the same sample. <https://www.nature.com/articles/s41598-018-27757-8>

Therefore the two reverse ones are: `ACTCCTACGGGAGGCAGCA` and `ATTAGAWACCCBDGTAGTCC`. We use the locations relative to a reference, if the local alignment search fails (https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0076185 for V1V2 location, for instance).

In [17]:
hyper_types = ["16Sv1v2", "16Sv3v4"]
#start = res.end_query - res.len_query
# v1234[] contains primers for v1v2 forward, v1v2 rev complement, v3v4 forward, v3v4 rev complem 
v1234 = ["AGAGTTTGATCCTGGCTCAG","ACTCCTACGGGAGGCAGCA","CCTACGGGAGGCAGCAG","ATTAGAWACCCBDGTAGTCC"];
# fallback: use relative position

hypervar = np.array([135, 360, 540, 820, 900, 1100, 1240, 1390, 1500]) # last position, including assumed 16S size
hypervar = hypervar/hypervar[-1] # fractions, w.r.t sequence length

def get_hypervar_16S (sequence):
    seqsize = len(sequence) 
    n_h = [int(x * seqsize) for x in hypervar[:4]] # fallback 
    res = [parasail.sw_stats_diag_16(x,sequence,9,1,parasail.blosum30) for x in v1234]

    v1 = res[0].end_ref - res[0].end_query # first position of match
    v2 = res[1].end_ref + 1 # last position of match is end_ref, which shold be included (thus, +1)
    if (v1 < 0):
        v1 = 0 
    v3 = res[2].end_ref - res[2].end_query # first position of match
    v4 = res[3].end_ref + 1 # last position of match is end_ref, which shold be included (thus, +1)
    if (v2 - v1 < 100) and (v4 - v3 > 100): ## only v3v4 was found
        v1 = 0; v2 = v3 + 20 ## primers have 20 bp of overlap
    if (v2 - v1 > 100) and (v4 - v3 < 100): ## only v1v2 was found
        v3 = v2 - 20; v4 = v3 + 460 
    
    if (v2 - v1 < 100): # last resource, use relative position
        v1 = 0; v2 = n_h[1]
    if (v4 - v3 < 100):
        v3 = n_h[1]; v4 = n_h[3] ## start of hypervar v3 and end of v4
    if (v4 > seqsize):
        v4 = seqsize
    
    print ("DEBUG:", v1, v2, v3, v4)
    return {"16Sv1v2":sequence[v1:v2], "16Sv3v4":sequence[v3:v4]}

In [ ]:
[i.seq for i in rna_concat["16v1v2"] if "09585" in i.id]

In [7]:
def align_seqs_small (unaligned):
    """ muscle alignment using pipes (not files) """
    cline = Align.Applications.MuscleCommandline(clwstrict=False, fasta=True, diags=True, maxiters=2, maxhours=4.) 
    child = subprocess.Popen(str(cline), stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE, 
                             universal_newlines=True, shell=(sys.platform!="win32"))

    SeqIO.write(unaligned, child.stdin, "fasta") # after this muscle is still waiting, so...
    child.stdin.close() # ... we must close the handle by hand, which will then make muscle start calculations
    aligned = AlignIO.read(child.stdout, "fasta")  # read from stdout as a fasta file 
    child.terminate()
    return aligned

def align_seqs (sequences=None, maxiters=12, infile=None, outfile=None, mafft = True):
    print ("started aligning...", flush=True, end=" ")
    if (sequences is None) and (infile is None):
        print ("ERROR: You must give me an alignment object or file")
        return [] ## OTOH if both are present then infile is overwritten with contents of sequences[]
    if infile is None:
        ifl = "/tmp/in.fas"
    else:
        ifl = infile
    if outfile is None:
        ofl = "/tmp/out.fas"
    else:
        ofl = outfile
    SeqIO.write(sequences, ifl, "fasta")
    if (mafft is False):
        proc_run = subprocess.check_output("muscle -in " + ifl + " -diags -maxiters " + str(maxiters) + " -out " + ofl,
                                       shell=True, universal_newlines=True)
    else: # "--parttree --6merpair" and 0.123 is to avoid very long alignments
        proc_run = subprocess.check_output("mafft --ep 0.3 --op 3.0 --auto " + ifl + " > " + ofl,
                                       shell=True, universal_newlines=True)
      
    aligned = AlignIO.read(ofl, "fasta")
    print ("Finished",flush=True)
    if infile is None:
        os.system("rm -f " + ifl)
    if outfile is None:
        os.system("rm -f " + ofl)
    return aligned

def ambiguous_consensus_from_alignment (align):
    #summary_align = AlignInfo.SummaryInfo(Align.MultipleSeqAlignment(align))
    summary_align = AlignInfo.SummaryInfo(align) # must be aligned sequences
    pssm = summary_align.pos_specific_score_matrix()
    consensus = []; 
    for score in pssm: # something like {'A': 0, 'T': 4.0, 'G': 0, 'C': 2.0} for each column
        base_present = ''.join(sorted([base for base in "ACGT" if score[base] > 0])) ## neglect indels
        if (base_present): 
            consensus.append(iupac_dna[base_present])
    return ''.join(consensus)

def get_longest_from_dict(rRNA):
    longest_dict = {}
    for rna in rrna_types:
        longest_dict[rna] = str(rRNA[rna][ np.argsort([len(x) for x in rRNA[rna]])[0] ].seq)
    return longest_dict

def get_consensus_from_dict(rRNA):
    consensus_dict = {}
    for rna in rrna_types:
        consensus_dict[rna] = ambiguous_consensus_from_alignment(align_seqs_small(rRNA[rna]))
    return consensus_dict

## RefSeq reading function
- receives element from our table (but uses only the sequence name) and returns a dictionary with sequences for every RNA found

In [8]:
def get_rrna_from_genbank_to_dict (table_fs):
    gbank = SeqIO.parse(gzip.open(table_fs[0], "rt"), "genbank")
    rRNA = {}
    for rna in rrna_types:
        rRNA[rna] = []
    genome = next(gbank) # no point in iterating over gbank, only this has whole information
    for feature in genome.features:
        if(feature.type == "rRNA"):
            try:
                feature_name = feature.qualifiers['product'][0].upper()
            except KeyError:
                print("WARNING: no 'product' feature in" + table_fs[0])
            for rna in rrna_types:
                if rna in feature_name:
                    this_product = rna
                    desc = feature.qualifiers['locus_tag'][0]
                    seq = feature.extract(genome.seq)
                    record = SeqRecord(seq, id=desc)
                    rRNA[this_product].append(record)
                    break
    success = True
    for rna in rrna_types:
        success = success and rRNA[rna]
    if success:
        return rRNA
    return None

In [12]:
## run this if you want to reset variables
rna_concat = {}
rna_consen = {}
stats_dict = {}
for rna in rrna_types+hyper_types: # must also initalise 'v1v2' and 'v3v4'
    rna_concat[rna] = []
    rna_consen[rna] = []
for rna in rrna_types:
    stats_dict["Longest " + rna] = []
    stats_dict["Consensus " + rna] = []
    stats_dict["Copies " + rna] = []

## Main loop, reading through all sequences in table
- table is the one produced above, with refseq genome file path and distinct taxonomic classifications
- this loop will create:
    1. `rna_concat[]` and `rna_consen[]` dictionaries with longest and consensus, respect., sequences from each "valid" genomes. the keys of dictionary are 16S, 23S, 5S, 16Sv1v2 etc. 
    2. new table with "valid" genomes only
    3. dictionary with statistics, that  will used by Panda to save spreadsheet 
- "valid" genomes are those for which reasonable RNA sequences were found (some are not annotated, some are missing,..), although these genomes were already curated by GTDB
- this loops processes 40 genomes/minute (i.e. this cell takes ~20 minutes to complete)

In [18]:
all_fl_sp = []

for file_counter, tfs in enumerate(tbl_fl_sp):
    if not (file_counter+1)%20:
        print (str(file_counter+1), end=" ", flush=True)
    rna_dict = get_rrna_from_genbank_to_dict (tfs)
    # (some dict will have no usable features or b/c too long/too short
    if rna_dict: # tfs[2] = GTDB, 3=lsuSILVA, 5=NCBI, 6=ssuSILVA
        spname = species_name_from_binomial (tfs[2], tfs[1]) # tfs[1] = accession number 
        
        concat_seqs = get_longest_from_dict (rna_dict)
        concat_seqs.update(get_hypervar_16S(concat_seqs['16S'])) # adds two elements, with v4 and v1v2 regions
        for rna, rseq in concat_seqs.items():
            rna_concat[rna].append(SeqRecord(Seq.Seq(rseq,Alphabet.IUPAC.ambiguous_dna),id=spname,description=spname))
        
        consen_seqs = get_consensus_from_dict (rna_dict)
        consen_seqs.update(get_hypervar_16S(consen_seqs['16S'])) # adds two elements, with v4 and v1v2 regions
        for rna, rseq in consen_seqs.items():
            rna_consen[rna].append(SeqRecord(Seq.Seq(rseq,Alphabet.IUPAC.ambiguous_dna),id=spname,description=spname))
        
        for rna in rrna_types: # stored into CSV 
            stats_dict["Longest " + rna].append(len(concat_seqs[rna]))
            stats_dict["Consensus " + rna].append(len(consen_seqs[rna]))
            stats_dict["Copies " + rna].append(len(rna_dict[rna])) # how many copies
        all_fl_sp.append(tfs) # successful elements of tbl_fs_sp    

# save table
fl=gzip.open(outdir+"all_fl_sp.pickle.gz", "w"); pickle.dump([all_fl_sp],fl,2); fl.close()

print ("") # newline
for rna in rrna_types:
    x = stats_dict["Longest " + rna] 
    prtnbr = [np.mean(x), np.std(x), len(x), np.min(x), np.percentile(x, 5), np.percentile(x, 95), np.max(x)]
    prtstr = " ".join(["{0:0.2f}".format(i) for i in prtnbr])
    print_redblack (rna, "        ", prtstr);
    x = stats_dict["Consensus " + rna]
    prtnbr = [np.mean(x), np.std(x), len(x), np.min(x), np.percentile(x, 5), np.percentile(x, 95), np.max(x)]
    prtstr = " ".join(["{0:0.2f}".format(i) for i in prtnbr])
    print_redblack ("consensus", "  ", prtstr);

DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 350 334 800
DEBUG: 7 350 334 800
DEBUG: 7 352 336 802
DEBUG: 7 352 336 802
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 7 350 334 800
DEBUG: 7 350 334 800
DEBUG: 0 164 144 611
DEBUG: 7 351 335 802
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 7 356 340 805
DEBUG: 7 356 340 806
DEBUG: 7 352 336 802
DEBUG: 7 352 336 802
DEBUG: 7 350 334 797
DEBUG: 7 352 336 802
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 7 351 335 801
DEBUG: 7 352 336 802
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
20 DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 7 352 336 802
DEBUG: 7 3

DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 7 350 334 800
DEBUG: 7 350 334 800
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
200 DEBUG: 6 362 346 812
DEBUG: 6 362 346 812
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 9 365 349 815
DEBUG: 6 362 346 812
DEBUG: 6 362 346 812
DEBUG: 9 357 341 807
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 7 350 334 800
DEBUG: 7 350 334 800
DEBUG: 9 358 342 808
DEBUG: 9 358 342 809
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 7 356 340 806
DEBUG: 7 356 340 806
DEBUG: 7 356 340 805
DEBUG: 7 356 340 806
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 9 358 342 808
DEBUG: 8 364 348 814
DEBUG: 8 364 348 814
DEBUG: 6 362 346 812
DEBUG: 6 362 346 812
DEBUG: 7 363 347 813
DEBUG: 7 363 347 813
220 DEBUG

In [ ]:
## no need to run in general: reading from previous analysis
fl=gzip.open(outdir +  "all_fl_sp.pickle.gz", "r"); [all_fl_sp] = pickle.load(fl); fl.close()

## Save statistics as CSV spreadsheet
- create pandas.DataFrame

In [ ]:
def create_dataframe_from_table_filename_species (table_fs, stats_dictionary):
    dfd = {"Organism (NCBI)":[], "Accession Number":[],"File Name":[], "Species (GTDB)":[]}
    for elem in table_fs: # output at top of notebook has order of elem columns
        dfd["Accession Number"].append(elem[1]) 
        dfd["Organism (NCBI)"].append(elem[4])
        dfd["File Name"].append(elem[0].split("/")[-1])
        #dfd["Species (GTDB)"].append(" ".join(elem[3].split("_")))
        dfd["Species (GTDB)"].append(elem[2])
    for k,v in stats_dictionary.items(): ## append statistics 
        dfd[k] = v
    return pandas.DataFrame(data=dfd)

df = create_dataframe_from_table_filename_species (all_fl_sp, stats_dict)
df.to_csv(outdir + "all.csv")

## Align RNA genes independently
- that is, each gene or fragment (e.g. 16Sv1v2) is aligned and saved in fasta format to disk
- the auxiliary function above is used to call MAFFT (with e=0.123 for similar sequences)
- this step usually takes ~5 minutes (for 700 sequences)

In [19]:
# align each of the dictionary values (lists of unaligned seqs)
concat_aligned = {k:align_seqs(sequences=v, infile=outdir + k + "_long.unfas", outfile=outdir + k + "_long.fasta") for k,v in rna_concat.items()}
consen_aligned = {k:align_seqs(sequences=v, infile=outdir + k + "_consensus.unfas", outfile=outdir + k + "_consensus.fasta") for k,v in rna_consen.items()}

started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished
started aligning... Finished


## Auxiliary functions for concatenating alignments

In [ ]:
## rna_alignment is a dict with one alignment per rRNA; below the dictionaries have _sequence_ids_ as keys.
def add_missing_sequences (align_1, align_2):
    seqdict_1 = SeqIO.to_dict(align_1); seqdict_2 = SeqIO.to_dict(align_2)
    n1 = align_1.get_alignment_length(); n2 = align_2.get_alignment_length();
    for k in seqdict_1:
        if k not in seqdict_2:
            seqdict_2[k] = SeqRecord(Seq.Seq('N'*n2, Alphabet.IUPAC.ambiguous_dna), id=k, description=k)
    for k in seqdict_2:
        if k not in seqdict_1:
            seqdict_1[k] = SeqRecord(Seq.Seq('N'*n1, Alphabet.IUPAC.ambiguous_dna), id=k, description=k)
    return Align.MultipleSeqAlignment(seqdict_1.values()), Align.MultipleSeqAlignment(seqdict_2.values())

def create_species_genus_labels (seqlist, have_paralogs = False): 
    species = ['.'.join(sequence.id.split('.')[:2]) for sequence in seqlist]
    genus = [sequence.id.split('.')[0] for sequence in seqlist]
    if have_paralogs:# assumes 'genus_species_code_number' names 
        sample = ['.'.join(sequence.id.split('.')[:3]) for sequence in seqlist]
        return species, genus, sample
    return species, genus 

#### if resuming from previous analysis, run code below to read genome names table and alignments

In [ ]:
# DO NOT RUN::  read table
concat_aligned = {k:AlignIO.read(outdir + k + "_long.fasta", "fasta") for k in rrna_types+hyper_types}
consen_aligned = {k:AlignIO.read(outdir + k + "_consensus.fasta", "fasta") for k in rrna_types+hyper_types}

#### creates list of gene labels in a specific order

In [ ]:
# order of outfile_list is important, since defines order in table, figures
outfile_list = [outdir + str(keys) for keys in hyper_types + rrna_types] # 16S etc. but also v1v2 v4
print (outfile_list)

## Create concatenated alignments
- from single-gene alignments, merge two-by-two and also the three of them together (16Sv1v2 etc. alignments are not used, only the full 16S)
- if gene is missing, replace it by "N" (although this should not happen, since against our definition of "valid" genome above, in 'main loop')
- we read alignments from disk, merge, and save them to disk. No actual alignment optimisation takes place. 

In [ ]:
## (1) Individual rRNAs (already done by align_seqs)
for r1, r2 in itertools.combinations(rrna_types,2): # ensure all samples are present
    concat_aligned[r1], concat_aligned[r2] = add_missing_sequences(concat_aligned[r1], concat_aligned[r2])
    
## (2) pairs of rRNAs (16+23, 16+5, 23+5)
for r1, r2 in itertools.combinations(rrna_types,2):
    outfile_list.append(outdir + str(r1) + str(r2))
    for thisalign, suff in zip([concat_aligned, consen_aligned], ["_long", "_consensus"]):
        seq_dict_1 = SeqIO.to_dict(thisalign[r1])
        seq_dict_2 = SeqIO.to_dict(thisalign[r2])
        tmpalign = [SeqRecord(Seq.Seq(str(seq_dict_1[k].seq) + str(seq_dict_2[k].seq), 
                                      Alphabet.IUPAC.ambiguous_dna), id=k, description=k) for k in seq_dict_1]
        # tmpalign has concat seqs related to _same_ sample (that's why a dictionary and not simply align1+align2)
        output_handle = open (outdir + str(r1) + str(r2) + suff + ".fasta", "w")
        SeqIO.write(tmpalign, output_handle, "fasta")
        output_handle.close()

## (3) all three concatenated
r1, r2, r3 = rrna_types
outfile_list.append(outdir + str(r1) + str(r2) + str(r3))

for thisalign, suff in zip([concat_aligned, consen_aligned], ["_long", "_consensus"]):
    seq_dict_1 = SeqIO.to_dict(thisalign[r1])
    seq_dict_2 = SeqIO.to_dict(thisalign[r2])
    seq_dict_3 = SeqIO.to_dict(thisalign[r3])
    tmpalign = [SeqRecord(Seq.Seq(str(seq_dict_1[k].seq) + str(seq_dict_2[k].seq) + str(seq_dict_3[k].seq), 
                                  Alphabet.IUPAC.ambiguous_dna), id=k, description=k) for k in seq_dict_1]
    output_handle = open (outdir + str(r1) + str(r2) + str(r3) + suff + ".fasta", "w")
    SeqIO.write(tmpalign, output_handle, "fasta")
    output_handle.close()

## Infer maximum likelihood trees with iqtree
- consensus and long sequences are split in two cells just for convenience, since they may take a while

In [ ]:
for fname in [ofile + suff for suff in ["_consensus"] for ofile in outfile_list]:
    subprocess.check_output("iqtree -s " + fname + ".fasta -nt 8 -m HKY+G",shell=True,universal_newlines=True)

In [ ]:
for fname in [ofile + suff for suff in ["_long"] for ofile in outfile_list]:
    subprocess.check_output("iqtree -s " + fname + ".fasta -nt 8 -m HKY+G",shell=True,universal_newlines=True)

## next step: go to notebook 028.monophyly_scores.ipynb 
- Now that we have alignments and trees, we proceed to calculate monophyly scores